# Data Cleaning & Feature Engineering

## Handle Missing Values

In [1]:
# imports
import pandas as pd
import numpy as np

In [2]:
diabetes = pd.read_csv('diabetic_dropped.csv')
diabetes = diabetes.drop(columns={'Unnamed: 0'})

In [3]:
diabetes['race'].value_counts(normalize=True)*100

race
Caucasian          74.778413
AfricanAmerican    18.876639
?                   2.233555
Hispanic            2.001651
Other               1.479866
Asian               0.629876
Name: proportion, dtype: float64

In [4]:
# Impute with Unknown
diabetes['race'] = diabetes['race'].replace('?', 'Unknown')

In [5]:
# only 0.0029% of the rows (3 rows) are unknown/invalid for gender so we will drop them
diabetes['gender'].value_counts(normalize=True)*100

gender
Female             53.758623
Male               46.238429
Unknown/Invalid     0.002948
Name: proportion, dtype: float64

In [6]:
diabetes = diabetes[diabetes['gender']!='Unknown/Invalid']

In [7]:
print(f'? in diag_1: {(diabetes['diag_1'] == '?').sum()}')
print(f'? in diag_2: {(diabetes['diag_2'] == '?').sum()}')
print(f'? in diag_3: {(diabetes['diag_3'] == '?').sum()}')

? in diag_1: 21
? in diag_2: 358
? in diag_3: 1423


In [8]:
# drop the 21 rows that have a ? for the primary diagnosis
diabetes = diabetes[diabetes['diag_1'] != '?']

# replace ? with no diagnosis since we're assuming some encounters didn't have 2nd and 3rd diagnoses, rather than missing data
diabetes['diag_2'] = diabetes['diag_2'].replace('?', 'nodiag2')
diabetes['diag_3'] = diabetes['diag_3'].replace('?', 'nodiag3')
diabetes.head()

,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,change,diabetesMed,readmitted
0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,...,No,No,No,No,No,No,No,No,No,0
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,No,No,No,Up,No,No,No,Ch,Yes,0
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,No,No,No,No,No,No,No,No,Yes,0
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,No,No,No,Up,No,No,No,Ch,Yes,0
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,No,No,No,Steady,No,No,No,Ch,Yes,0


In [9]:
# replace NaNs in max_glu_serum with the string category name 'not tested'
diabetes['max_glu_serum'] = diabetes['max_glu_serum'].replace(np.nan, 'Not tested')

In [10]:
# create a feature to note if max_glu_serum was tested or not
diabetes['glu_tested'] = (diabetes['max_glu_serum']!='Not tested').astype(int)

In [11]:
# replace NaNs in A1Cresult with the string category name 'not tested'
diabetes['A1Cresult'] = diabetes['A1Cresult'].replace(np.nan, 'Not tested')

# create a feature to note if A1Cresult was tested or not
diabetes['A1C_tested'] = (diabetes['A1Cresult']!='Not tested').astype(int)
diabetes.head()

,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,change,diabetesMed,readmitted,glu_tested,A1C_tested
0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,...,No,No,No,No,No,No,No,0,0,0
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,No,Up,No,No,No,Ch,Yes,0,0,0
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,No,No,No,No,No,No,Yes,0,0,0
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,No,Up,No,No,No,Ch,Yes,0,0,0
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,No,Steady,No,No,No,Ch,Yes,0,0,0


# Encode Categorical Variables

## One-Hot Encoding

In [12]:
diabetes['race'].value_counts(normalize=True)

race
Caucasian          0.747892
AfricanAmerican    0.188722
Unknown            0.022302
Hispanic           0.020002
Other              0.014792
Asian              0.006290
Name: proportion, dtype: float64

In [13]:
diabetes['race_Caucasian'] = (diabetes['race'] == 'Caucasian').astype(int)
diabetes['race_AA'] = (diabetes['race'] == 'AfricanAmerican').astype(int)
diabetes['race_unknown'] = (diabetes['race'] == 'Unknown').astype(int)
diabetes['race_Hispanic'] = (diabetes['race'] == 'Hispanic').astype(int)
diabetes['race_other'] = (diabetes['race'] == 'Other').astype(int)
diabetes['race_Asian'] = (diabetes['race'] == 'Asian').astype(int)

In [14]:
diabetes['gender'].unique()

array(['Female', 'Male'], dtype=object)

In [15]:
# one-hot encoding gender
diabetes['gender_male'] = (diabetes['gender'] == 'Male').astype(int)
diabetes['gender_female'] = (diabetes['gender'] == 'Female').astype(int)

In [16]:
# one-hot encoding admission types since they are nominal, not ordinal
diabetes['admission_type_id'].unique()

array([6, 1, 2, 3, 4, 5, 8, 7])

In [17]:
admission_type_map = {
    1: 'Emergency',
    2: 'Urgent',
    3: 'Elective',
    4: 'Newborn',
    5: 'Not Available',
    6: 'NULL',
    7: 'Trauma Center',
    8: 'Not Mapped'
}

diabetes['admission_type'] = diabetes['admission_type_id'].map(admission_type_map)

# replace nulls
diabetes['admission_type'] = diabetes['admission_type'].replace(['Not Available', 'NULL', 'Not Mapped'], 'Unknown')

# one-hot encoding
admission_dummies = pd.get_dummies(diabetes['admission_type'], prefix='admission_type')
diabetes = pd.concat([diabetes, admission_dummies], axis=1)
diabetes.head()


,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,race_Asian,gender_male,gender_female,admission_type,admission_type_Elective,admission_type_Emergency,admission_type_Newborn,admission_type_Trauma Center,admission_type_Unknown,admission_type_Urgent
0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,...,0,0,1,Unknown,False,False,False,False,True,False
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,0,0,1,Emergency,False,True,False,False,False,False
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,0,0,1,Emergency,False,True,False,False,False,False
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,0,1,0,Emergency,False,True,False,False,False,False
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,0,1,0,Emergency,False,True,False,False,False,False


In [18]:
pd.DataFrame(diabetes['discharge_disposition_id'].value_counts(normalize=True).iloc[0:10]).reset_index().sort_values(by=\
    'discharge_disposition_id')

,discharge_disposition_id,proportion
0,1,0.591948
4,2,0.020906
1,3,0.137112
9,4,0.008010
7,5,0.011618
2,6,0.126772
6,11,0.016139
3,18,0.036268
5,22,0.019579
8,25,0.009711


In [19]:
discharge_status_dict = {
    1: "Discharged to home",
    2: "Discharged/transferred to another short term hospital",
    3: "Discharged/transferred to SNF",
    4: "Discharged/transferred to ICF",
    5: "Discharged/transferred to another type of inpatient care institution",
    6: "Discharged/transferred to home with home health service",
    11: "Expired",
    18: "NULL",
    22: "Discharged/transferred to another rehab fac including rehab units of a hospital.",
    25: "Not Mapped"
}

diabetes['discharge_status'] = diabetes['discharge_disposition_id'].map(discharge_status_dict)

# replace nulls
diabetes['discharge_status'] = diabetes['discharge_status'].replace(['NULL', 'Not Mapped'], 'Unknown')
diabetes['discharge_status'] = diabetes['discharge_status'].fillna('Other')

# one-hot encoding
discharge_dummies2 = pd.get_dummies(diabetes['discharge_status'], prefix='discharge_status')
diabetes = pd.concat([diabetes, discharge_dummies2], axis=1)
diabetes.head()


,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,discharge_status_Discharged to home,discharge_status_Discharged/transferred to ICF,discharge_status_Discharged/transferred to SNF,discharge_status_Discharged/transferred to another rehab fac including rehab units of a hospital.,discharge_status_Discharged/transferred to another short term hospital,discharge_status_Discharged/transferred to another type of inpatient care institution,discharge_status_Discharged/transferred to home with home health service,discharge_status_Expired,discharge_status_Other,discharge_status_Unknown
0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,...,False,False,False,False,False,False,False,False,False,True
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,True,False,False,False,False,False,False,False,False,False
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,True,False,False,False,False,False,False,False,False,False
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,True,False,False,False,False,False,False,False,False,False
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,True,False,False,False,False,False,False,False,False,False


In [20]:
pd.DataFrame(diabetes['admission_source_id'].value_counts(normalize=True).iloc[0:10]).reset_index().sort_values(by='admission_source_id')

,admission_source_id,proportion
1,1,0.290480
5,2,0.010851
7,3,0.001838
3,4,0.031315
6,5,0.008394
4,6,0.022252
0,7,0.564998
9,9,0.001229
2,17,0.066639
8,20,0.001582


In [21]:
admission_source_dict = {
    1: "Physician Referral",
    2: "Clinic Referral",
    3: "HMO Referral",
    4: "Transfer from a hospital",
    5: "Transfer from a Skilled Nursing Facility (SNF)",
    6: "Transfer from another health care facility",
    7: "Emergency Room",
    9: "Not Available",
    17: "NULL",
    20: "Not Mapped"
}


diabetes['admission_source'] = diabetes['admission_source_id'].map(admission_source_dict)

# replace nulls
diabetes['admission_source'] = diabetes['admission_source'].replace(['Not Available', 'NULL', 'Not Mapped'], 'Unknown')
diabetes['admission_source'] = diabetes['admission_source'].fillna('Other')

# one-hot encoding
admission_dummies = pd.get_dummies(diabetes['admission_source'], prefix='admission_source')
diabetes = pd.concat([diabetes, admission_dummies], axis=1)
diabetes.head()


,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,...,admission_source,admission_source_Clinic Referral,admission_source_Emergency Room,admission_source_HMO Referral,admission_source_Other,admission_source_Physician Referral,admission_source_Transfer from a Skilled Nursing Facility (SNF),admission_source_Transfer from a hospital,admission_source_Transfer from another health care facility,admission_source_Unknown
0,8222157,Caucasian,Female,[0-10),6,25,1,1,41,0,...,Physician Referral,False,False,False,False,True,False,False,False,False
1,55629189,Caucasian,Female,[10-20),1,1,7,3,59,0,...,Emergency Room,False,True,False,False,False,False,False,False,False
2,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,11,5,...,Emergency Room,False,True,False,False,False,False,False,False,False
3,82442376,Caucasian,Male,[30-40),1,1,7,2,44,1,...,Emergency Room,False,True,False,False,False,False,False,False,False
4,42519267,Caucasian,Male,[40-50),1,1,7,1,51,0,...,Emergency Room,False,True,False,False,False,False,False,False,False


In [22]:
# one-hot encoding change
diabetes['change_yes'] = (diabetes['change'] == 'Ch').astype(int)
diabetes['change_no'] = (diabetes['change'] == 'No').astype(int)

In [23]:
# one-hot encoding change
diabetes['diabetesMed_yes'] = (diabetes['diabetesMed'] == 'Yes').astype(int)
diabetes['diabetesMed_no'] = (diabetes['diabetesMed'] == 'No').astype(int)

## Label Encoding

In [24]:
diabetes['age'].unique()

array(['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)',
       '[60-70)', '[70-80)', '[80-90)', '[90-100)'], dtype=object)

In [25]:
# label encoding for age
age_order = ['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)']
diabetes['age'] = pd.Categorical(diabetes['age'], categories=age_order, ordered=True)
diabetes['age_encoding'] = diabetes['age'].cat.codes
diabetes['age_encoding']

0         0
1         1
2         2
3         3
4         4
         ..
101761    7
101762    8
101763    7
101764    8
101765    7
Name: age_encoding, Length: 101742, dtype: int8

In [26]:
diabetes['max_glu_serum'].unique()

array(['Not tested', '>300', 'Norm', '>200'], dtype=object)

In [27]:
# label encoding for max_glu_serum
glucose_order = ['Not tested', 'Norm', '>200', '>300'] # least to most severe
diabetes['max_glu_serum'] = pd.Categorical(diabetes['max_glu_serum'], categories=glucose_order, ordered=True)
diabetes['glu_serum_encoding'] = diabetes['max_glu_serum'].cat.codes
diabetes['glu_serum_encoding']

0         0
1         0
2         0
3         0
4         0
         ..
101761    0
101762    0
101763    0
101764    0
101765    0
Name: glu_serum_encoding, Length: 101742, dtype: int8

In [28]:
diabetes['A1Cresult'].unique()

array(['Not tested', '>7', '>8', 'Norm'], dtype=object)

In [29]:
# label encoding for A1C
A1C_order = ['Not tested', 'Norm', '>7', '>8']
diabetes['A1Cresult'] = pd.Categorical(diabetes['A1Cresult'], categories=A1C_order, ordered=True)
diabetes['A1C_encoded'] = diabetes['A1Cresult'].cat.codes
diabetes['A1C_encoded']

0         0
1         0
2         0
3         0
4         0
         ..
101761    3
101762    0
101763    0
101764    0
101765    0
Name: A1C_encoded, Length: 101742, dtype: int8

In [30]:
diabetes.columns

Index(['patient_nbr', 'race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'num_lab_procedures', 'num_procedures', 'num_medications',
       'number_outpatient', 'number_emergency', 'number_inpatient', 'diag_1',
       'diag_2', 'diag_3', 'number_diagnoses', 'max_glu_serum', 'A1Cresult',
       'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
       'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'change', 'diabetesMed', 'readmitted',
       'glu_tested', 'A1C_tested', 'race_Caucasian', 'race_AA', 'race_unknown',
       'race_Hispanic', 'race_other', 'race_Asian', 'gender_male',
       'gender_female', 'admission_type', 'admission_type_Elective',
       'admission_type_Emergency', 'a

In [31]:
# drop columns now that i have completed encoding
drop_cols = ['race', 'gender', 'admission_type', 'discharge_status', 'max_glu_serum', 'A1Cresult', 'age', 'change',\
              'diabetesMed', 'patient_nbr', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id',\
                'diag_1', 'diag_2', 'diag_3', 'admission_source']
diabetes = diabetes.drop(columns=drop_cols)

In [32]:
diabetes.columns

Index(['time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'metformin', 'repaglinide',
       'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide',
       'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone',
       'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone', 'tolazamide',
       'insulin', 'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'readmitted', 'glu_tested', 'A1C_tested',
       'race_Caucasian', 'race_AA', 'race_unknown', 'race_Hispanic',
       'race_other', 'race_Asian', 'gender_male', 'gender_female',
       'admission_type_Elective', 'admission_type_Emergency',
       'admission_type_Newborn', 'admission_type_Trauma Center',
       'admission_type_Unknown', 'admission_type_Urgent',
       'discharge_status_Discharged to home',
       'discharge_status_Discharged/transferred to ICF',
       'disc

In [33]:
diabetes['readmitted'].unique()

array([0, 1])

## Feature Engineering

In [34]:

diabetes['total_visits'] = diabetes['number_outpatient'] + diabetes['number_emergency'] + diabetes['number_inpatient']

In [35]:
diabetes['polypharmacy'] = (diabetes['num_medications'] > 5).astype(int)
diabetes['polypharmacy']

0         0
1         1
2         1
3         1
4         1
         ..
101761    1
101762    1
101763    1
101764    1
101765    0
Name: polypharmacy, Length: 101742, dtype: int64

In [36]:
med_cols = [
    'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
    'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
    'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
    'miglitol', 'troglitazone', 'tolazamide', 'insulin',
    'glyburide-metformin', 'glipizide-metformin', 'glimepiride-pioglitazone'
]

diabetes['num_meds_chanaged'] = diabetes[med_cols].apply(
    lambda row: sum(val in ['Up', 'Down'] for val in row), axis=1
)


In [37]:
diabetes.to_csv('cleaned_diabetes.csv')